In [ ]:
%load_ext autoreload
%autoreload 2

import geopandas as gpd
import line_geom_graph as lgg

In [ ]:
streets = gpd.read_file('data_intermediate/edge_bearings.gpkg')

In [ ]:
streets_prj = streets.to_crs(9498)

In [ ]:
lgg.validate_lines_gdf(streets_prj)

In [ ]:
gdf = streets_prj[['geometry']].copy()

In [ ]:
import shapely

In [ ]:
shapely.get_coords( gdf.geometry.values )

In [ ]:
A0 = shapely.get_point(gdf.geometry.values, 0)

In [ ]:
A1 = shapely.get_point(gdf.geometry.values, 1)

In [ ]:
import numpy as np

In [ ]:
bearings_planar

In [ ]:
def bearings_planar(points_a, points_b):
    # points_a, points_b: array-like of Shapely Points (same length)
    A = shapely.get_coordinates(points_a)   # shape (N, 2) -> [x, y]
    B = shapely.get_coordinates(points_b)
    dx = B[:, 0] - A[:, 0]
    dy = B[:, 1] - A[:, 1]
    # bearing: 0° = North, 90° = East
    brg = (np.degrees(np.arctan2(dx, dy)) + 360.0) % 360.0
    # optional: set NaN where points coincide
    brg[(dx == 0) & (dy == 0)] = np.nan
    return brg

def half_edges( gdf ):

    he_gdfs = []

    for he in ['a','b']:
        if he == 'a':
            p0 = shapely.get_point(gdf.geometry.values, 0)
            p1 = shapely.get_point(gdf.geometry.values, 1)
        elif he == 'b':
            p0 = shapely.get_point(gdf.geometry.values, -1)
            p1 = shapely.get_point(gdf.geometry.values, -2)
            
        he_gdf = gpd.GeoDataFrame(
            data = pd.DataFrame({'bearing':bearings_planar(p0, p1)}),
            geometry = p0
        )

        he_gdf['id'] = gdf.index
        he_gdf['he'] = he
        he_gdf['he_id'] = he_gdf['id'].astype(str) + he_gdf['he']

        he_gdfs.append( he_gdf )

    return pd.concat( he_gdfs )

In [ ]:
he_gdf = half_edges( gdf )
he_gdf

In [ ]:
he_self_join = he_gdf[['id','he_id','bearing','geometry']].sjoin(
    he_gdf[['id','he_id','bearing','geometry']]
).drop(columns='index_right')

In [ ]:
he_self_join

In [ ]:
# remove self-joins
he_join = he_self_join[ he_self_join['he_id_left'] != he_self_join['he_id_right'] ].copy()

In [ ]:
he_graph = he_join[ he_join['he_id_left'] < he_join['he_id_right'] ].copy()
he_graph

In [ ]:
def bearing_change_from_opposite(a, b, mode="abs"):
    """
    a, b: bearings in degrees (arrays or scalars).
    Reverse 'a' then compute change from a_rev to b.

    mode:
      - "signed" -> minimal signed change in [-180, 180)
      - "abs"    -> magnitude of minimal change in [0, 180]
      - "cw"     -> clockwise change in [0, 360)
      - "ccw"    -> counter-clockwise change in [0, 360)
    """
    a = np.asarray(a, dtype=float)
    b = np.asarray(b, dtype=float)
    a_rev = (a + 180.0) % 360.0

    if mode == "signed":
        return ((b - a_rev + 540.0) % 360.0) - 180.0
    elif mode == "abs":
        return np.abs(((b - a_rev + 540.0) % 360.0) - 180.0)
    elif mode == "cw":
        return (b - a_rev) % 360.0
    elif mode == "ccw":
        return (a_rev - b) % 360.0
    else:
        raise ValueError("mode must be one of: 'signed', 'abs', 'cw', 'ccw'")

In [ ]:
he_graph['bc'] = bearing_change_from_opposite(he_graph['bearing_left'], he_graph['bearing_right'])

In [ ]:
he_graph

### Save graph

In [ ]:
he_graph[['id_left','id_right','bc']]

In [ ]:
he_graph['bc'].plot(
    kind='hist',
    bins=18
)

In [ ]:
num_bins = 18

# Split bins in half to prevent bin-edge effects around common values.
# Bins will be merged in pairs after the histogram is computed. The last
# bin edge is the same as the first (i.e., 0 degrees = 360 degrees).
num_split_bins = num_bins * 2
split_bin_edges = np.linspace(0, 360, num_split_bins + 1)
bearings = he_graph['bc']
split_bin_counts, split_bin_edges = np.histogram(
    bearings,
    bins=split_bin_edges,
)
# Move last bin to front, so eg 0.01 degrees and 359.99 degrees will be
# binned together. Then combine counts from pairs of split bins.
split_bin_counts = np.roll(split_bin_counts, 1)
bin_counts = split_bin_counts[::2] + split_bin_counts[1::2]
# Every other edge of the split bins is the center of a merged bin.
bin_centers = split_bin_edges[range(0, num_split_bins - 1, 2)]

bin_counts, bin_centers

In [ ]:
import networkx as nx

In [ ]:
MG = nx.MultiGraph()

In [ ]:
he_graph.head(2)

In [ ]:
MG = nx.from_pandas_edgelist(
    he_graph,
    source='he_id_left',
    target='he_id_right',
    edge_attr='bc',
    create_using=nx.MultiGraph()
)

In [ ]:
import osmnx as ox

In [ ]:
from matplotlib.projections.polar import PolarAxes
from typing import Any
from typing import overload
from typing import Literal
from matplotlib.figure import Figure  # noqa: TC002
from matplotlib.axes._axes import Axes  # noqa: TC002
import matplotlib.pyplot as plt

mpl_available = True

def _verify_mpl() -> None:
    """Verify that matplotlib is installed and imported."""
    if not mpl_available:  # pragma: no cover
        msg = "matplotlib must be installed as an optional dependency for visualization."
        raise ImportError(msg)

# if polar = False, return Axes
@overload
def _get_fig_ax(
    ax: Axes | None,
    figsize: tuple[float, float],
    bgcolor: str | None,
    polar: Literal[False],
) -> tuple[Figure, Axes]: ...


# if polar = True, return PolarAxes
@overload
def _get_fig_ax(
    ax: Axes | None,
    figsize: tuple[float, float],
    bgcolor: str | None,
    polar: Literal[True],
) -> tuple[Figure, PolarAxes]: ...


def _get_fig_ax(
    ax: Axes | None,
    figsize: tuple[float, float],
    bgcolor: str | None,
    polar: bool,  # noqa: FBT001
) -> tuple[Figure, Axes | PolarAxes]:
    """
    Generate a matplotlib Figure and (Polar)Axes or return existing ones.

    Parameters
    ----------
    ax
        If not None, plot on this pre-existing axes instance.
    figsize
        If `ax` is None, create new figure with size `(width, height)`.
    bgcolor
        Background color of figure.
    polar
        If True, generate a `PolarAxes` instead of an `Axes` instance.

    Returns
    -------
    fig, ax
        The resulting matplotlib figure and axes objects.
    """
    if ax is None:
        if polar:
            # make PolarAxes
            fig, ax = plt.subplots(figsize=figsize, subplot_kw={"projection": "polar"})
        else:
            # make regular Axes
            fig, ax = plt.subplots(figsize=figsize, facecolor=bgcolor, frameon=False)
            ax.set_facecolor(bgcolor)
    else:
        fig = ax.figure  # type: ignore[assignment]

    return fig, ax

def plot_orientation(  # noqa: PLR0913
    G: nx.MultiGraph | nx.MultiDiGraph,
    *,
    num_bins: int = 36,
    min_length: float = 0,
    weight: str | None = None,
    ax: PolarAxes | None = None,
    figsize: tuple[float, float] = (5, 5),
    area: bool = True,
    color: str = "#003366",
    edgecolor: str = "k",
    linewidth: float = 0.5,
    alpha: float = 0.7,
    title: str | None = None,
    title_y: float = 1.05,
    title_font: dict[str, Any] | None = None,
    xtick_font: dict[str, Any] | None = None,
    bin_counts = None,
    bin_centers = None,
) -> tuple[Figure, PolarAxes]:
    """
    Plot a polar histogram of a spatial network's edge bearings.

    Ignores self-loop edges as their bearings are undefined. If `G` is a
    MultiGraph, all edge bearings will be bidirectional (ie, two reciprocal
    bearings per undirected edge). If `G` is a MultiDiGraph, all edge bearings
    will be directional (ie, one bearing per directed edge). See also the
    `bearings` module.

    For more info see: Boeing, G. 2019. "Urban Spatial Order: Street Network
    Orientation, Configuration, and Entropy." Applied Network Science, 4 (1),
    67. https://doi.org/10.1007/s41109-019-0189-1

    Parameters
    ----------
    G
        Unprojected graph with `bearing` attributes on each edge.
    num_bins
        Number of bins. For example, if `num_bins=36` is provided, then each
        bin will represent 10 degrees around the compass.
    min_length
        Ignore edges with "length" attribute values less than `min_length`.
    weight
        If not None, weight the edges' bearings by this (non-null) edge
        attribute.
    ax
        If not None, plot on this pre-existing axes instance (must have
        projection=polar).
    figsize
        If `ax` is None, create new figure with size `(width, height)`.
    area
        If True, set bar length so area is proportional to frequency.
        Otherwise, set bar length so height is proportional to frequency.
    color
        Color of the histogram bars.
    edgecolor
        Color of the histogram bar edges.
    linewidth
        Width of the histogram bar edges.
    alpha
        Opacity of the histogram bars.
    title
        The figure's title.
    title_y
        The y position to place `title`.
    title_font
        The title's `fontdict` to pass to matplotlib.
    xtick_font
        The xtick labels' `fontdict` to pass to matplotlib.

    Returns
    -------
    fig, ax
        The resulting matplotlib figure and polar axes objects.
    """
    _verify_mpl()

    if title_font is None:
        title_font = {"family": "DejaVu Sans", "size": 24, "weight": "bold"}
    if xtick_font is None:
        xtick_font = {
            "family": "DejaVu Sans",
            "size": 10,
            "weight": "bold",
            "alpha": 1.0,
            "zorder": 3,
        }

    ## get the bearing distribution's bin counts and center values in degrees
    #bin_counts, bin_centers = bearing._bearings_distribution(
    #    G,
    #    num_bins,
    #    min_length=min_length,
    #    weight=weight,
    #)

    # positions: where to center each bar
    positions = np.deg2rad(bin_centers)

    # width: make bars fill the circumference without gaps or overlaps
    width = 2 * np.pi / num_bins

    # radius: how long to make each bar. set bar length so either the bar area
    # (ie, via sqrt) or the bar height is proportional to the bin's frequency
    bin_frequency = bin_counts / bin_counts.sum()
    radius = np.sqrt(bin_frequency) if area else bin_frequency

    # create PolarAxes (if not passed-in) then set N at top and go clockwise
    fig, ax = _get_fig_ax(ax=ax, figsize=figsize, bgcolor=None, polar=True)
    ax.set_theta_zero_location("N")
    ax.set_theta_direction("clockwise")
    ax.set_ylim(top=radius.max())

    # configure the y-ticks and remove their labels
    ax.set_yticks(np.linspace(0, radius.max(), 5))
    ax.set_yticklabels(labels="")

    # configure the x-ticks and their labels
    xticklabels = ["N", "", "E", "", "S", "", "W", ""]
    ax.set_xticks(ax.get_xticks())
    ax.set_xticklabels(labels=xticklabels, fontdict=xtick_font)
    ax.tick_params(axis="x", which="major", pad=-2)

    # draw the bars
    ax.bar(
        positions,
        height=radius,
        width=width,
        align="center",
        bottom=0,
        zorder=2,
        color=color,
        edgecolor=edgecolor,
        linewidth=linewidth,
        alpha=alpha,
    )

    if title:
        ax.set_title(title, y=title_y, fontdict=title_font)
    fig.tight_layout()
    return fig, ax

In [ ]:
MG = nx.from_pandas_edgelist(
    he_graph,
    source='he_id_left',
    target='he_id_right',
    edge_attr='bc',
    create_using=nx.MultiGraph()
)

In [ ]:
plot_orientation(
    MG,
    bin_counts=bin_counts,
    bin_centers=bin_centers
)